# Hyperparameter grid search
NB the input data to the DNN is not normalised.

Hyperparameter grid search adapted from Machine Learning Mastery
https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
Using scikit-learn to grid search the batch size and epochs

In [ ]:
import sys
from pathlib import Path
from datetime import datetime
from dateutil.tz import gettz

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras import utils
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import GridSearchCV

np.random.seed(757566)

# User inputs

In [ ]:
fname = 'private_dog0_correct_plus' # private_dog0_correct_plus

logs_dir = '../logs'
timestamp = '{:%Y-%m-%dT%H:%M}'.format(datetime.now(gettz("Europe/London")))
logs_dir = logs_dir +'/' + timestamp
if 'private' in fname:
    fdir = '../data/private_data/private_events_dev2' 
else:
    fdir = '../data' 

# Utilities

In [ ]:
def readucr(filename):
    ''' Load a dataset from a file in UCR format
    space delimited, class labels in the first column.
    Returns
    X : DNN input data
    Y : class labels
    '''
    data = np.loadtxt(Path(filename))
    Y = data[:,0]
    X = data[:,1:]
    return X, Y


def prepare_data(y):
    ''' Return y as a categorical array'''
    nb_classes = 2
    y = (y - y.min())/(y.max()-y.min())*(nb_classes-1)
    Y = utils.to_categorical(y, nb_classes)
    return Y

# Create model

In [ ]:
def create_resnet(input_shape=(0,0), num_features0=-1, num_features1=-1, filter_size=-1, pooling_size=-1, dropout=-1):
    ''' Return ResNet model '''
    nb_classes = 2
    print(input_shape, num_features0, num_features1, filter_size, pooling_size, dropout)
    
    # Preparation block
    x = Input(shape=(input_shape))
    conv = keras.layers.Conv1D(num_features0, filter_size, padding='same')(x)
    conv = keras.layers.BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    conv = keras.layers.MaxPooling1D(pooling_size)(conv)
    
    # First block
    skip = conv
    conv = keras.layers.Conv1D(num_features0, filter_size, padding='same')(conv)
    conv = keras.layers.BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    
    conv = keras.layers.Conv1D(num_features0, filter_size, padding='same')(conv)
    conv = keras.layers.BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    
    conv = keras.layers.Conv1D(num_features1, filter_size, padding='same')(conv)
    conv = keras.layers.BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    
    conv = keras.layers.Conv1D(num_features1, filter_size, padding='same')(conv)
    conv = keras.layers.BatchNormalization()(conv)
    shortcut = keras.layers.Conv1D(num_features1, filter_size, padding='same')(skip)
    shortcut = keras.layers.BatchNormalization()(shortcut)
    conv = keras.layers.add([conv, shortcut])
    conv = Activation('relu')(conv)
    
    # Second block
    skip = conv
    conv = keras.layers.Conv1D(num_features0, filter_size, padding='same')(conv)
    conv = keras.layers.BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    
    conv = keras.layers.Conv1D(num_features0, filter_size, padding='same')(conv)
    conv = keras.layers.BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    
    conv = keras.layers.Conv1D(num_features1, filter_size, padding='same')(conv)
    conv = keras.layers.BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    
    conv = keras.layers.Conv1D(num_features1, filter_size, padding='same')(conv)
    conv = keras.layers.BatchNormalization()(conv)
    shortcut = keras.layers.Conv1D(num_features1, filter_size, padding='same')(skip)
    shortcut = keras.layers.BatchNormalization()(shortcut)
    conv = keras.layers.add([conv, shortcut])
    conv = Activation('relu')(conv)
    
    # Third block
    skip = conv
    conv = keras.layers.Conv1D(num_features0*2, filter_size, padding='same')(conv)
    conv = keras.layers.BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    
    conv = keras.layers.Conv1D(num_features0*2, filter_size, padding='same')(conv)
    conv = keras.layers.BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    
    conv = keras.layers.Conv1D(num_features1*2, filter_size, padding='same')(conv)
    conv = keras.layers.BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    
    conv = keras.layers.Conv1D(num_features1*2, filter_size, padding='same')(conv)
    conv = keras.layers.BatchNormalization()(conv)
    shortcut = keras.layers.Conv1D(num_features1*2, filter_size, padding='same')(skip)
    shortcut = keras.layers.BatchNormalization()(shortcut)
    conv = keras.layers.add([conv, shortcut])
    conv = Activation('relu')(conv)
    
    # Output block
    full = keras.layers.GlobalAveragePooling1D()(conv)
    y = Dropout(dropout, name='Dropout')(full)
    out = Dense(nb_classes, activation='sigmoid')(full)
    model = Model(x, out)
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    print('Model compiled')
    return model
    

# Run

In [ ]:
# load dataset
x_train, y_train = readucr(fdir+'/'+fname+'/'+fname+'_TRAIN.txt')
x_test, y_test = readucr(fdir+'/'+fname+'/'+fname+'_TEST.txt')
X = np.concatenate((x_train, x_test), axis=0)
Y = np.concatenate((y_train, y_test), axis=0)
X = X.reshape(X.shape + (1,))
input_shape = X.shape[1:]
print(input_shape)
Y = prepare_data(Y)

# define the grid search parameters
batch_size = 32
epochs = 1000
num_features0 = [64, 128, 256]
num_features1 = [64, 128, 256]
filter_size = [2, 8, 32]
pooling_size = [4, 8, 16]
dropout = [0.2, 0.5]
param_grid = dict(num_features0=num_features0, num_features1=num_features1, 
                  filter_size=filter_size, pooling_size=pooling_size, 
                  dropout=dropout,)

# Create model and run the grid search
if True:
    model = KerasClassifier(build_fn=create_resnet, 
                            input_shape=input_shape,
                            batch_size=batch_size, epochs=epochs,
                            verbose=1)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, error_score=0, return_train_score=True)
    grid_result = grid.fit(X, Y)

    # Summarise results
    print('Best score:', grid_result.best_score_, 'using: ', grid_result.best_params_)
    cv = pd.DataFrame(grid_result.cv_results_)
    pd.set_option('display.max_colwidth', -1)
    cv[['mean_test_score', 'std_test_score', 'params']]
else:
    x_train = x_train.reshape(x_train.shape + (1,))
    x_test = x_test.reshape(x_test.shape + (1,))
    Y_train = prepare_data(y_train)
    Y_test = prepare_data(y_test)

    callbacks = []
    tensorboard_dir = '../logs/tensorboard'
    tensorboard_dir = tensorboard_dir +'/' + timestamp
    tb_dir = tensorboard_dir+'/'+fname
    Path(tb_dir).mkdir(parents=True, exist_ok=True) 
    callbacks.append(keras.callbacks.TensorBoard(log_dir=tb_dir, histogram_freq=0))
    model = create_resnet(input_shape, num_features0[0], num_features1, filter_size, pooling_size, dropout)
    hist = model.fit(x_train, Y_train, batch_size=batch_size, epochs=epochs,
                      verbose=1, validation_data=(x_test, Y_test), callbacks=callbacks)
       

In [ ]:
cv

In [ ]:
print('Completed at', '{:%Y-%m-%dT%H:%M}'.format(datetime.now(gettz("Europe/London"))))

In [ ]:
Path(logs_dir+'/'+fname).mkdir(parents=True, exist_ok=True)
filename = Path(logs_dir+'/'+fname+'/resnet_grid_search_summary.csv')
cv.to_csv(filename)
print('Results saved to', filename)